# Automated Metadata Extraction for Better Retrieval + Synthesis

In this tutorial, we show you how to perform automated metadata extraction for better retrieval results.
We use two extractors: a QuestionAnsweredExtractor which generates question/answer pairs from a piece of text, and also a SummaryExtractor which extracts summaries, not only within the current text, but also within adjacent texts.

We show that this allows for "chunk dreaming" -each individual chunk to have more "holistic" details, leading to higher answer quality given retrieved results.

Our data source is taken from Eugene Yan's popular article on LLM Patterns: https://eugeneyan.com/writing/llm-patterns/

## Setup

In [59]:
import nest_asyncio

nest_asyncio.apply()

import os
import openai

In [60]:
# OPTIONAL: setup W&B callback handling for tracing
from llama_index import set_global_handler

set_global_handler("wandb", run_args={"project": "llamaindex"})

In [18]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"
openai.api_key = os.environ["OPENAI_API_KEY"]

## Define Metadata Extractors

Here we define metadata extractors. We define two variants:
- metadata_extractor_1 only contains the QuestionsAnsweredExtractor
- metadata_extractor_2 contains both the QuestionsAnsweredExtractor as well as the SummaryExtractor

In [61]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.schema import MetadataMode

In [62]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

We also show how to instantiate the `SummaryExtractor` and `QuestionsAnsweredExtractor`.

In [6]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=256, chunk_overlap=128)


metadata_extractor_1 = MetadataExtractor(
    extractors=[
        QuestionsAnsweredExtractor(questions=3, llm=llm),
    ],
    in_place = False
)

metadata_extractor = MetadataExtractor(
    extractors=[
        SummaryExtractor(summaries=["prev", "self", "next"], llm=llm),
        QuestionsAnsweredExtractor(questions=3, llm=llm),
    ],
    in_place = False
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    # metadata_extractor=metadata_extractor,
)

## Load in Data, Run Extractors

We load in Eugene's essay (https://eugeneyan.com/writing/llm-patterns/) using our LlamaHub SimpleWebPageReader.

We then run our extractors.

In [7]:
from llama_index import SimpleDirectoryReader

In [9]:
# load in blog

from llama_hub.web.simple_web.base import SimpleWebPageReader

reader = SimpleWebPageReader(html_to_text=True)
docs = reader.load_data(urls=["https://eugeneyan.com/writing/llm-patterns/"])

In [ ]:
print(docs[0].get_content())

In [12]:
orig_nodes = node_parser.get_nodes_from_documents(docs)

In [13]:
# take just the first 8 nodes for testing
nodes = orig_nodes[20:28]

In [14]:
print(nodes[3].get_content(metadata_mode="all"))

is to measure the distance that words would
have to move to convert one sequence to another.

However, there are several pitfalls to using these conventional benchmarks and
metrics.

First, there’s **poor correlation between these metrics and human judgments.**
BLEU, ROUGE, and others have had [negative correlation with how humans
evaluate fluency](https://arxiv.org/abs/2008.12009). They also showed moderate
to less correlation with human adequacy scores. In particular, BLEU and ROUGE
have [low correlation with tasks that require creativity and
diversity](https://arxiv.org/abs/2303.16634).

Second, these metrics often have **poor adaptability to a wider variety of
tasks**. Adopting a metric proposed for one task to another is not always
prudent. For example, exact match metrics such as BLEU and ROUGE are a poor
fit for tasks like abstractive summarization or dialogue. Since they’re based
on n-gram overlap between output and reference, they don’t make sense for a
dialogue task where a w

### Run metadata extractors

In [15]:
# process nodes with metadata extractor
nodes_1 = metadata_extractor_1.process_nodes(nodes)

Extracting questions:   0%|          | 0/8 [00:00<?, ?it/s]

In [16]:
print(nodes_1[3].get_content(metadata_mode="all"))

[Excerpt from document]
questions_this_excerpt_can_answer: 1. What is the correlation between conventional metrics like BLEU and ROUGE and human judgments of fluency and adequacy in natural language processing tasks?
2. How well do metrics like BLEU and ROUGE perform in tasks that require creativity and diversity?
3. Why are exact match metrics like BLEU and ROUGE not suitable for tasks like abstractive summarization or dialogue?
Excerpt:
-----
is to measure the distance that words would
have to move to convert one sequence to another.

However, there are several pitfalls to using these conventional benchmarks and
metrics.

First, there’s **poor correlation between these metrics and human judgments.**
BLEU, ROUGE, and others have had [negative correlation with how humans
evaluate fluency](https://arxiv.org/abs/2008.12009). They also showed moderate
to less correlation with human adequacy scores. In particular, BLEU and ROUGE
have [low correlation with tasks that require creativity and


In [17]:
# 2nd pass: run summaries, and then metadata extractor

# process nodes with metadata extractor
nodes_2 = metadata_extractor.process_nodes(nodes)

Extracting summaries:   0%|          | 0/8 [00:00<?, ?it/s]

Extracting questions:   0%|          | 0/8 [00:00<?, ?it/s]

### Visualize some sample data

In [18]:
print(nodes_2[3].get_content(metadata_mode="all"))

[Excerpt from document]
prev_section_summary: The section discusses the comparison between BERTScore and MoverScore, two metrics used to evaluate the quality of text generation models. MoverScore is described as a metric that measures the effort required to transform one text sequence into another by mapping semantically related words. The section also highlights the limitations of conventional benchmarks and metrics, such as poor correlation with human judgments and low correlation with tasks requiring creativity.
next_section_summary: The section discusses the limitations of current evaluation metrics in natural language processing tasks. It highlights three main issues: lack of creativity and diversity in metrics, poor adaptability to different tasks, and poor reproducibility. The section mentions specific metrics like BLEU and ROUGE, and also references studies that have reported high variance in metric scores.
section_summary: The section discusses the limitations of conventional 

In [54]:
print(nodes_2[1].get_content(metadata_mode="all"))

[Excerpt from document]
prev_section_summary: The section discusses the F_{BERT} formula used in BERTScore and highlights the advantages of BERTScore over simpler metrics like BLEU and ROUGE. It also introduces MoverScore, another metric that uses contextualized embeddings but allows for many-to-one matching. The key topics are BERTScore, MoverScore, and the differences between them.
next_section_summary: The section discusses the comparison between BERTScore and MoverScore, two metrics used to evaluate the quality of text generation models. MoverScore is described as a metric that measures the effort required to transform one text sequence into another by mapping semantically related words. The section also highlights the limitations of conventional benchmarks and metrics, such as poor correlation with human judgments and low correlation with tasks requiring creativity.
section_summary: The key topics of this section are BERTScore and MoverScore, which are methods used to compute the 

## Setup RAG Query Engines, Compare Results! 

We setup 3 indexes/query engines on top of the three node variants.

In [20]:
from llama_index import VectorStoreIndex
from llama_index.response.notebook_utils import display_source_node, display_response

In [21]:
# try out different query engines

# index0 = VectorStoreIndex(orig_nodes)
# index1 = VectorStoreIndex(nodes_1 + orig_nodes[8:])
# index2 = VectorStoreIndex(nodes_2 + orig_nodes[8:]) 

index0 = VectorStoreIndex(orig_nodes)
index1 = VectorStoreIndex(orig_nodes[:20] + nodes_1 + orig_nodes[28:])
index2 = VectorStoreIndex(orig_nodes[:20] + nodes_2 + orig_nodes[28:]) 

wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.


In [22]:
query_engine0 = index0.as_query_engine(similarity_top_k=1)
query_engine1 = index1.as_query_engine(similarity_top_k=1)
query_engine2 = index2.as_query_engine(similarity_top_k=1)

### Try out some questions

In this question, we see that the naive response `response0` only mentions BLEU and ROUGE, and lacks context about other metrics.

`response2` on the other hand has all metrics within its context.

In [23]:
# query_str = "In the original RAG paper, can you describe the two main approaches for generation and compare them?"
query_str = "Can you describe metrics for evaluating text generation quality, compare them, and tell me about their downsides"

response0 = query_engine0.query(query_str)
response1 = query_engine1.query(query_str)
response2 = query_engine2.query(query_str)

wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.


In [24]:
display_response(response0, source_length=1000, show_source=True, show_source_metadata=True)

**`Final Response:`** Metrics for evaluating text generation quality can vary depending on the task at hand. However, some commonly used metrics include BLEU, ROUGE, and exact match metrics. These metrics are often criticized for their poor adaptability to a wider variety of tasks. For example, exact match metrics like BLEU and ROUGE may not be suitable for tasks such as abstractive summarization or dialogue, where a wide range of responses are possible. This is because these metrics rely on n-gram overlap between the generated text and a reference, which may not capture the quality of the response accurately.

In addition to poor adaptability, these metrics also suffer from poor reproducibility. Different studies have reported high variance in the scores, which could be attributed to variations in human judgment collection or metric parameter settings. This means that the same metric may produce different results in different studies, making it challenging to compare and interpret the quality of text generation across different experiments.

Overall, while metrics like BLEU, ROUGE, and exact match metrics provide a quantitative way to evaluate text generation quality, they have limitations in terms of adaptability to different tasks and reproducibility of results. It is important to consider these downsides when using these metrics and to explore alternative approaches for evaluating text generation quality.

---

**`Source Node 1/1`**

**Node ID:** c588b330-1f15-41af-a9ff-d04aaf295e94<br>**Similarity:** 0.8403638027791802<br>**Text:** require creativity and
diversity](https://arxiv.org/abs/2303.16634).

Second, these metrics often have **poor adaptability to a wider variety of
tasks**. Adopting a metric proposed for one task to another is not always
prudent. For example, exact match metrics such as BLEU and ROUGE are a poor
fit for tasks like abstractive summarization or dialogue. Since they’re based
on n-gram overlap between output and reference, they don’t make sense for a
dialogue task where a wide variety of responses are possible. An output can
have zero n-gram overlap with the reference but yet be a good response.

Third, these metrics have **poor reproducibility**. Even for the same metric,
[high variance is reported across different
studies](https://arxiv.org/abs/2008.12009), possibly due to variations in
human judgment collection or metric parameter settings. Another study of
[ROUGE scores](https://aclanthology.org/2023.acl-long.107/) across 2,000
studies found that scores were hard...<br>**Metadata:** {}<br>

In [91]:
print(response0.source_nodes[0].node.get_content())

require creativity and
diversity](https://arxiv.org/abs/2303.16634).

Second, these metrics often have **poor adaptability to a wider variety of
tasks**. Adopting a metric proposed for one task to another is not always
prudent. For example, exact match metrics such as BLEU and ROUGE are a poor
fit for tasks like abstractive summarization or dialogue. Since they’re based
on n-gram overlap between output and reference, they don’t make sense for a
dialogue task where a wide variety of responses are possible. An output can
have zero n-gram overlap with the reference but yet be a good response.

Third, these metrics have **poor reproducibility**. Even for the same metric,
[high variance is reported across different
studies](https://arxiv.org/abs/2008.12009), possibly due to variations in
human judgment collection or metric parameter settings. Another study of
[ROUGE scores](https://aclanthology.org/2023.acl-long.107/) across 2,000
studies found that scores were hard


In [26]:
display_response(response1, source_length=1000, show_source=True, show_source_metadata=True)

**`Final Response:`** Metrics for evaluating text generation quality can vary depending on the task at hand. However, some commonly used metrics include BLEU, ROUGE, and exact match metrics. These metrics are often criticized for their poor adaptability to a wider variety of tasks. For example, exact match metrics like BLEU and ROUGE may not be suitable for tasks such as abstractive summarization or dialogue, where a wide range of responses are possible. This is because these metrics rely on n-gram overlap between the generated text and a reference, which may not capture the quality of the response accurately.

In addition to poor adaptability, these metrics also suffer from poor reproducibility. Different studies have reported high variance in the scores, possibly due to variations in human judgment collection or metric parameter settings. This means that the same metric can produce different results in different studies, making it challenging to compare and interpret the quality of text generation across different experiments.

Overall, while metrics like BLEU, ROUGE, and exact match metrics provide a quantitative way to evaluate text generation quality, they have limitations in terms of adaptability to different tasks and reproducibility of results. Researchers are actively exploring alternative metrics and evaluation methods to address these downsides and provide more comprehensive and reliable assessments of text generation quality.

---

**`Source Node 1/1`**

**Node ID:** c588b330-1f15-41af-a9ff-d04aaf295e94<br>**Similarity:** 0.8353130580892497<br>**Text:** require creativity and
diversity](https://arxiv.org/abs/2303.16634).

Second, these metrics often have **poor adaptability to a wider variety of
tasks**. Adopting a metric proposed for one task to another is not always
prudent. For example, exact match metrics such as BLEU and ROUGE are a poor
fit for tasks like abstractive summarization or dialogue. Since they’re based
on n-gram overlap between output and reference, they don’t make sense for a
dialogue task where a wide variety of responses are possible. An output can
have zero n-gram overlap with the reference but yet be a good response.

Third, these metrics have **poor reproducibility**. Even for the same metric,
[high variance is reported across different
studies](https://arxiv.org/abs/2008.12009), possibly due to variations in
human judgment collection or metric parameter settings. Another study of
[ROUGE scores](https://aclanthology.org/2023.acl-long.107/) across 2,000
studies found that scores were hard...<br>**Metadata:** {'questions_this_excerpt_can_answer': '1. What are some limitations of using exact match metrics like BLEU and ROUGE for tasks such as abstractive summarization or dialogue?\n2. Why is it not always prudent to adopt a metric proposed for one task to another?\n3. What are some challenges in reproducing and comparing metric scores across different studies?'}<br>

In [25]:
display_response(response2, source_length=1000, show_source=True, show_source_metadata=True)

**`Final Response:`** Metrics for evaluating text generation quality include BERTScore and MoverScore. BERTScore measures the similarity between generated text and reference text using contextual embeddings, while MoverScore measures the effort required to transform one text sequence into another by mapping semantically related words. 

However, there are downsides to using conventional benchmarks and metrics. One downside is the poor correlation between these metrics and human judgments. Metrics like BLEU and ROUGE have been found to have low correlation with human evaluations of fluency and adequacy, as well as tasks requiring creativity and diversity. 

Another downside is the poor adaptability of these metrics to different tasks. Metrics like BLEU and ROUGE, which rely on n-gram overlap between output and reference, are not suitable for tasks like abstractive summarization or dialogue. Adopting a metric proposed for one task to another may not always be appropriate. 

Overall, these limitations highlight the need for more diverse and adaptable metrics in evaluating text generation quality.

---

**`Source Node 1/1`**

**Node ID:** 704a96bb-8bae-48cd-96f3-eba6c0bed37b<br>**Similarity:** 0.8393108424346125<br>**Text:** is to measure the distance that words would
have to move to convert one sequence to another.

However, there are several pitfalls to using these conventional benchmarks and
metrics.

First, there’s **poor correlation between these metrics and human judgments.**
BLEU, ROUGE, and others have had [negative correlation with how humans
evaluate fluency](https://arxiv.org/abs/2008.12009). They also showed moderate
to less correlation with human adequacy scores. In particular, BLEU and ROUGE
have [low correlation with tasks that require creativity and
diversity](https://arxiv.org/abs/2303.16634).

Second, these metrics often have **poor adaptability to a wider variety of
tasks**. Adopting a metric proposed for one task to another is not always
prudent. For example, exact match metrics such as BLEU and ROUGE are a poor
fit for tasks like abstractive summarization or dialogue. Since they’re based
on n-gram overlap between output and reference, they don’t make sense for a
dialogue task where ...<br>**Metadata:** {'prev_section_summary': 'The section discusses the comparison between BERTScore and MoverScore, two metrics used to evaluate the quality of text generation models. MoverScore is described as a metric that measures the effort required to transform one text sequence into another by mapping semantically related words. The section also highlights the limitations of conventional benchmarks and metrics, such as poor correlation with human judgments and low correlation with tasks requiring creativity.', 'next_section_summary': 'The section discusses the limitations of current evaluation metrics in natural language processing tasks. It highlights three main issues: lack of creativity and diversity in metrics, poor adaptability to different tasks, and poor reproducibility. The section mentions specific metrics like BLEU and ROUGE, and also references studies that have reported high variance in metric scores.', 'section_summary': 'The section discusses the limitations of conventional benchmarks and metrics used to measure the distance between word sequences. It highlights two main issues: poor correlation with human judgments and poor adaptability to different tasks. The metrics like BLEU and ROUGE have been found to have low correlation with human evaluations of fluency and adequacy, as well as tasks requiring creativity and diversity. Additionally, these metrics are not suitable for tasks like abstractive summarization or dialogue due to their reliance on n-gram overlap.', 'questions_this_excerpt_can_answer': '1. What are the limitations of conventional benchmarks and metrics in measuring the distance between word sequences?\n2. How do metrics like BLEU and ROUGE correlate with human judgments of fluency and adequacy?\n3. Why are metrics like BLEU and ROUGE not suitable for tasks like abstractive summarization or dialogue?'}<br>

In this next question, we ask about BERTScore/MoverScore. 

The responses are similar. But `response2` gives slightly more detail than `response0` since it has more information about MoverScore contained in the Metadata.

In [55]:
# query_str = "What are some reproducibility issues with the ROUGE metric? Give some details related to benchmarks and also describe other ROUGE issues. "
query_str = "Can you give a high-level overview of BERTScore/MoverScore + formulas if available?"

response0 = query_engine0.query(query_str)
response1 = query_engine1.query(query_str)
response2 = query_engine2.query(query_str)

wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.


In [56]:
display_response(response0, source_length=1000, show_source=True, show_source_metadata=True)

**`Final Response:`** BERTScore is a metric that is useful because it can account for synonyms and paraphrasing, unlike simpler metrics like BLEU and ROUGE. It uses contextualized embeddings to compute the distance between tokens in the generated output and reference. The formula for BERTScore is:

\[ F_{\text{BERT}} = \frac{2 \cdot P_{\text{BERT}} \cdot R_{\text{BERT}}}{P_{\text{BERT}} + R_{\text{BERT}}} \]

MoverScore, on the other hand, also uses contextualized embeddings to compute the distance between tokens in the generated output and reference. However, unlike BERTScore, which is based on one-to-one matching of tokens, MoverScore allows for many-to-one matching. This is also known as "soft alignment". Unfortunately, the formula for MoverScore is not provided in the given context.

---

**`Source Node 1/1`**

**Node ID:** ee11e571-14f3-49b2-baad-de21452f6bd1<br>**Similarity:** 0.8392526815428412<br>**Text:** = F_{\text{BERT}} =
\frac{2 \cdot P_{\text{BERT}} \cdot R_{\text{BERT}}}{P_{\text{BERT}} +
R_{\text{BERT}}}\\]

BERTScore is useful because it can account for synonyms and paraphrasing.
Simpler metrics like BLEU and ROUGE can’t do this due to their reliance on
exact matches. BERTScore has been shown to have better correlation for tasks
such as image captioning and machine translation.

**[MoverScore](https://arxiv.org/abs/1909.02622)** also uses contextualized
embeddings to compute the distance between tokens in the generated output and
reference. But unlike BERTScore, which is based on one-to-one matching (or
“hard alignment”) of tokens, MoverScore allows for many-to-one matching (or
“soft alignment”).

![BERTScore \(left\) vs. MoverScore...<br>**Metadata:** {}<br>

In [57]:
display_response(response1, source_length=1000, show_source=True, show_source_metadata=True)

**`Final Response:`** BERTScore is a metric that is useful because it can account for synonyms and paraphrasing, unlike simpler metrics like BLEU and ROUGE. It uses contextualized embeddings to compute the distance between tokens in the generated output and reference. The formula for BERTScore is:

\[ F_{\text{BERT}} = \frac{2 \cdot P_{\text{BERT}} \cdot R_{\text{BERT}}}{P_{\text{BERT}} + R_{\text{BERT}}} \]

MoverScore, on the other hand, also uses contextualized embeddings to compute the distance between tokens in the generated output and reference. However, unlike BERTScore, which is based on one-to-one matching of tokens, MoverScore allows for many-to-one matching. Unfortunately, the formula for MoverScore is not provided in the given context.

---

**`Source Node 1/1`**

**Node ID:** ee11e571-14f3-49b2-baad-de21452f6bd1<br>**Similarity:** 0.8483994150419983<br>**Text:** = F_{\text{BERT}} =
\frac{2 \cdot P_{\text{BERT}} \cdot R_{\text{BERT}}}{P_{\text{BERT}} +
R_{\text{BERT}}}\\]

BERTScore is useful because it can account for synonyms and paraphrasing.
Simpler metrics like BLEU and ROUGE can’t do this due to their reliance on
exact matches. BERTScore has been shown to have better correlation for tasks
such as image captioning and machine translation.

**[MoverScore](https://arxiv.org/abs/1909.02622)** also uses contextualized
embeddings to compute the distance between tokens in the generated output and
reference. But unlike BERTScore, which is based on one-to-one matching (or
“hard alignment”) of tokens, MoverScore allows for many-to-one matching (or
“soft alignment”).

![BERTScore \(left\) vs. MoverScore...<br>**Metadata:** {'questions_this_excerpt_can_answer': '1. What is the advantage of using BERTScore over simpler metrics like BLEU and ROUGE?\n2. How does MoverScore differ from BERTScore in terms of token matching?\n3. What tasks have shown better correlation with BERTScore, such as image captioning and machine translation?'}<br>

In [58]:
display_response(response2, source_length=1000, show_source=True, show_source_metadata=True)

**`Final Response:`** BERTScore is a method used to compute the similarity between generated output and reference in tasks like image captioning and machine translation. It uses one-to-one matching of tokens to measure the similarity. The formula used in BERTScore is F_{BERT} = \frac{2 \cdot P_{BERT} \cdot R_{BERT}}{P_{BERT} + R_{BERT}}, where P_{BERT} represents precision and R_{BERT} represents recall.

MoverScore, on the other hand, also uses contextualized embeddings to compute the distance between tokens in the generated output and reference. Unlike BERTScore, which uses one-to-one matching, MoverScore allows for many-to-one matching. It solves an optimization problem to measure the distance that words would have to move to convert one sequence to another. Unfortunately, the specific formula for MoverScore is not provided in the given context.

---

**`Source Node 1/1`**

**Node ID:** ee11e571-14f3-49b2-baad-de21452f6bd1<br>**Similarity:** 0.8476886116109771<br>**Text:** = F_{\text{BERT}} =
\frac{2 \cdot P_{\text{BERT}} \cdot R_{\text{BERT}}}{P_{\text{BERT}} +
R_{\text{BERT}}}\\]

BERTScore is useful because it can account for synonyms and paraphrasing.
Simpler metrics like BLEU and ROUGE can’t do this due to their reliance on
exact matches. BERTScore has been shown to have better correlation for tasks
such as image captioning and machine translation.

**[MoverScore](https://arxiv.org/abs/1909.02622)** also uses contextualized
embeddings to compute the distance between tokens in the generated output and
reference. But unlike BERTScore, which is based on one-to-one matching (or
“hard alignment”) of tokens, MoverScore allows for many-to-one matching (or
“soft alignment”).

![BERTScore \(left\) vs. MoverScore...<br>**Metadata:** {'next_section_summary': 'The key topics of this section are BERTScore and MoverScore, which are methods used to compute the similarity between generated output and reference in tasks like image captioning and machine translation. BERTScore uses one-to-one matching of tokens, while MoverScore allows for many-to-one matching. MoverScore solves an optimization problem to measure the distance that words would have to move to convert one sequence to another.', 'section_summary': 'The section discusses the F_{BERT} formula used in BERTScore and highlights the advantages of BERTScore over simpler metrics like BLEU and ROUGE. It also introduces MoverScore, another metric that uses contextualized embeddings but allows for many-to-one matching. The key topics are BERTScore, MoverScore, and the differences between them.', 'questions_this_excerpt_can_answer': '1. What is the key difference between BERTScore and MoverScore in terms of token matching?\n2. How does BERTScore account for synonyms and paraphrasing, unlike simpler metrics like BLEU and ROUGE?\n3. What are the advantages of using BERTScore over simpler metrics like BLEU and ROUGE in tasks such as image captioning and machine translation?'}<br>

In [79]:
response1.source_nodes[0].node.metadata

{}